In [1]:
import os
os.chdir("../") # Navigating to root directory)

In [2]:
from cnnClassifier.utils.common import load_env_variables_from_yaml

load_env_variables_from_yaml("env/env.yaml")

[2024-02-07 15:08:27,778: INFO : common: yaml file: env\env.yaml loaded sucessfully!]


In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.keras")

[2024-02-07 15:08:34,954: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-02-07 15:08:36,479: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-02-07 15:08:36,581: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaulationConfigEntity:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self)->EvaulationConfigEntity:
        eval_config = EvaulationConfigEntity(
            path_of_model=Path(f"{self.config.training.root_dir}/model.keras"),
            training_data=Path(f"{self.config.data_ingestion.unzip_dir}/Chest-CT-Scan-data"),
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [8]:
class EvaluationComponent:
    def __init__(self, config: EvaulationConfigEntity):
        self.config = config

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def _valid_gen(self):
        dataGen_kwargs = dict(
            rescale = 1.0/255,
            validation_split = 0.30
        )

        dataFlow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
            **dataGen_kwargs
        )

        self.valid_generator = valid_dataGen.flow_from_directory(
            directory = self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataFlow_kwargs
        )

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_gen()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy":self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = EvaluationComponent(config=eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-07 15:08:39,865: INFO : common: yaml file: config\config.yaml loaded sucessfully!]
[2024-02-07 15:08:39,880: INFO : common: yaml file: params.yaml loaded sucessfully!]
[2024-02-07 15:08:39,882: INFO : common: Created directory at: artifacts]
Found 102 images belonging to 2 classes.
[2024-02-07 15:08:42,909: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
[2024-02-07 15:08:43,154: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\engine\base_layer_utils.py:384: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
7/7 [==============================] - 10s 1s/step - loss: 27.4389 - accuracy: 0.5686
[2024-02-07 15:08:52,594: INFO : common: json file saved at: sc

2024/02/07 15:08:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-07 15:08:56,747: INFO : builder_impl: Assets written to: C:\Users\samtn\AppData\Local\Temp\tmpnx1dm3wt\model\data\model\assets]


c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/07 15:09:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
